In [ ]:
from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk("data/s2v_reddit_2019_lg")

In [ ]:
import spacy
from sense2vec import Sense2VecComponent

nlp = spacy.load("en_core_web_sm")
s2v = nlp.add_pipe("sense2vec")
s2v.from_disk("data/s2v_reddit_2019_lg")

In [ ]:
doc = nlp("PersonX accepts PersonY's resignation")
for phrase in doc._.s2v_phrases:
    print(phrase)

In [ ]:
len(doc._.s2v_phrases[1].vector)

In [ ]:
doc2 = nlp("car insurance")

In [1]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.optim import Adam
from tqdm import tqdm
from torch import nn
import wandb
import spacy
    
nlp = spacy.load("en_core_web_sm")
s2v = nlp.add_pipe("sense2vec")
s2v.from_disk("data/s2v_reddit_2019_lg")

class HeadDataset(Dataset):
    def __init__(self, df):
        self.labels = []
        self.embeddings = []

        for index, text in enumerate(df['text']):
            doc = nlp(text)
            embeddings = []
            for phrase in doc._.s2v_phrases:
                embeddings.append(phrase.vector)
            
            if embeddings:
                self.embeddings.append(np.mean(embeddings, axis=0))
                self.labels.append(df['label'][index])

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]



class MaxPool(nn.Module):
    def forward(self, X):
        values, _ = torch.max(X, dim=1)
        return values


class AvgPool(nn.Module):
    def forward(self, X):
        return torch.mean(X, dim=1)


class Sense2VecClassifier(nn.Module):

    def __init__(self, hidden_dim=128, num_classes=3, embedding_dim=96):
        super().__init__()
        self.linear1 = nn.Linear(embedding_dim, hidden_dim)
        self.activation1 = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, X):
        outputs = self.linear1(X)
        outputs = self.activation1(outputs)
        outputs = self.linear2(outputs)
        outputs = self.softmax(outputs)
        return outputs
    
    def save_pretrained(self, path):
        torch.save(self, path)


def train(model, train_dataset, val_dataset, learning_rate=1e-3, epochs=10, batch_size=8):
    # wandb.init(project="kogito-relation-matcher", config={"learning_rate": learning_rate, "epochs": epochs, "batch_size": batch_size})

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.NLLLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
        print("Using CUDA")

    model = model.to(device)
    criterion = criterion.to(device)

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
            model.zero_grad()

            train_label = train_label.to(device)
            X = train_input.to(device)

            output = model(X)
            
            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                X = val_input.to(device)

                output = model(X)

                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        train_loss = total_loss_train / len(train_dataset)
        train_acc = total_acc_train / len(train_dataset)
        val_loss = total_loss_val / len(val_dataset)
        val_acc = total_acc_val / len(val_dataset)

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {train_loss: .3f} \
            | Train Accuracy: {train_acc: .3f} \
            | Val Loss: {val_loss: .3f} \
            | Val Accuracy: {val_acc: .3f}')
        
        # wandb.log({"train_loss": train_loss, "train_accuracy": train_acc, "val_loss": val_loss, "val_accuracy": val_acc})
        # model.save_pretrained(f"./models/checkpoint_{epoch_num}.pth")

/root/.conda/envs/kogito/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-03-21 21:06:30.889120: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-21 21:06:30.889161: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from relation_modeling_utils import load_data

train_df = load_data("data/atomic2020_data-feb2021/train.tsv")
dev_df = load_data("data/atomic2020_data-feb2021/dev.tsv")
train_data = HeadDataset(train_df)
val_data = HeadDataset(dev_df)

In [ ]:
train_df['label'][0]

In [5]:
model = Sense2VecClassifier()
train(model=model, train_dataset=train_data, val_dataset=val_data, epochs=100, batch_size=128, learning_rate=1e-5)

Using CUDA


100%|██████████| 408/408 [00:00<00:00, 417.99it/s]


Epochs: 1 | Train Loss:  0.008             | Train Accuracy:  0.538             | Val Loss:  0.008             | Val Accuracy:  0.497


100%|██████████| 408/408 [00:00<00:00, 421.40it/s]


Epochs: 2 | Train Loss:  0.008             | Train Accuracy:  0.616             | Val Loss:  0.008             | Val Accuracy:  0.506


...

Epochs: 98 | Train Loss:  0.005             | Train Accuracy:  0.647             | Val Loss:  0.006             | Val Accuracy:  0.532


100%|██████████| 408/408 [00:00<00:00, 457.64it/s]


Epochs: 99 | Train Loss:  0.005             | Train Accuracy:  0.648             | Val Loss:  0.006             | Val Accuracy:  0.532


100%|██████████| 408/408 [00:00<00:00, 458.56it/s]


Epochs: 100 | Train Loss:  0.005             | Train Accuracy:  0.647             | Val Loss:  0.006             | Val Accuracy:  0.531
